<a href="https://colab.research.google.com/github/ECV21/R-coding-challenges/blob/main/Data_cleaning_and_manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data cleaning and Manipulation

To be efficient analysts, we should have good tools for reformatting dats for our needs so we can do our actual owrk like making plots and fitting models.

 To demonstrate, we’ll be working with dataset from ADF&G containing commercial catch data from 1878-1997. https://knb.ecoinformatics.org/#view/df35b.304.2.

#Carga de librería

In [ ]:
library(dplyr)
library(tidyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




#Carga de datos

In [ ]:
original_data <- read.csv(url("https://knb.ecoinformatics.org/knb/d1/mn/v2/object/df35b.302.1", method = "libcurl"),
                    stringsAsFactors = FALSE)
head(original_data)

,Region,Year,Chinook,Sockeye,Coho,Pink,Chum,All,notesRegCode
,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<chr>
1,SSE,1886,0,5,0,0,0,5,
2,SSE,1887,0,155,0,0,0,155,
3,SSE,1888,0,224,16,0,0,240,
4,SSE,1889,0,182,11,92,0,285,
5,SSE,1890,0,251,42,0,0,292,
6,SSE,1891,0,274,24,0,0,298,


#Selecting/removing columns

In [ ]:
#Let's select only the columns we want:

original_data %>%
  select(Region, Year, Chinook, Sockeye, Coho, Pink, Chum) %>%
  head()

,Region,Year,Chinook,Sockeye,Coho,Pink,Chum
,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>
1,SSE,1886,0,5,0,0,0
2,SSE,1887,0,155,0,0,0
3,SSE,1888,0,224,16,0,0
4,SSE,1889,0,182,11,92,0
5,SSE,1890,0,251,42,0,0
6,SSE,1891,0,274,24,0,0


In [ ]:
#SELECT allows you to say which columns you DON'T want, by passing unquoted column names preceded by minus(-) signs
original_data %>%
  select(-All, -notesRegCode) %>%
  head(15)

,Region,Year,Chinook,Sockeye,Coho,Pink,Chum
,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>
1,SSE,1886,0,5,0,0,0
2,SSE,1887,0,155,0,0,0
3,SSE,1888,0,224,16,0,0
4,SSE,1889,0,182,11,92,0
5,SSE,1890,0,251,42,0,0
6,SSE,1891,0,274,24,0,0
7,SSE,1892,0,207,11,8,0
8,SSE,1893,0,189,1,187,0
9,SSE,1894,0,253,5,529,0


In [ ]:
#Let's save our changes by overwritting the original_data variable:

original_data <- original_data %>%
  select(-All, -notesRegCode)

#Changing shape

In [ ]:
#Crear nuevas columnas: species, catch; ignorar Region and YEAR
original_data <- original_data %>%
  gather(species, catch, -Region, -Year)

head(original_data)

,Region,Year,species,catch
,<chr>,<int>,<chr>,<chr>
1,SSE,1886,Chinook,0
2,SSE,1887,Chinook,0
3,SSE,1888,Chinook,0
4,SSE,1889,Chinook,0
5,SSE,1890,Chinook,0
6,SSE,1891,Chinook,0


#Renaming columns

In [ ]:
#Let's rename the "catch" column to be called catch_thousands:
original_data1 <- original_data %>%
  rename(catch_thousands = catch)
head()

ERROR: Error in head.default(): argument "x" is missing, with no default


In [ ]:
head(original_data1)

,Region,Year,species,catch_thousands
,<chr>,<int>,<chr>,<chr>
1,SSE,1886,Chinook,0
2,SSE,1887,Chinook,0
3,SSE,1888,Chinook,0
4,SSE,1889,Chinook,0
5,SSE,1890,Chinook,0
6,SSE,1891,Chinook,0


#Adding columns

In [ ]:
#Let's create a new column called "catch" whit units of fish (instead of thousands of fish)

#ver tipo de datos:
summary(original_data1)



    Region               Year        species          catch_thousands   
 Length:8540        Min.   :1878   Length:8540        Length:8540       
 Class :character   1st Qu.:1922   Class :character   Class :character  
 Mode  :character   Median :1947   Mode  :character   Mode  :character  
                    Mean   :1946                                        
                    3rd Qu.:1972                                        
                    Max.   :1997                                        

In [ ]:
#la columna catch_thousands es tipo character, convertir a integers

catch_integers <- as.integer(original_data1$catch_thousands)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [ ]:
#R talks us that it couldn't convert ever value to an integer, for those values it couldn't convert, it put NA in its place.

#We can find put which values are NAs
which(is.na(catch_integers))

[1] 401

In [ ]:
#It looks like the 401st value is the proble,

original_data1[401,]

,Region,Year,species,catch_thousands
,<chr>,<int>,<chr>,<chr>
401,GSE,1955,Chinook,I


In [ ]:
original_data1 <- original_data1 %>%
  mutate(catch_thousands = ifelse(catch_thousands == "I", 1, catch_thousands),
         catch_thousands = as.integer(catch_thousands))

In [ ]:
#convertir en unidades:
original_data1 <- original_data1 %>%
  mutate(catch = catch_thousands * 1000)

summary(original_data1)


    Region               Year        species          catch_thousands  
 Length:8540        Min.   :1878   Length:8540        Min.   :    0.0  
 Class :character   1st Qu.:1922   Class :character   1st Qu.:    0.0  
 Mode  :character   Median :1947   Mode  :character   Median :   36.0  
                    Mean   :1946                      Mean   :  873.2  
                    3rd Qu.:1972                      3rd Qu.:  377.2  
                    Max.   :1997                      Max.   :53676.0  
     catch         
 Min.   :       0  
 1st Qu.:       0  
 Median :   36000  
 Mean   :  873249  
 3rd Qu.:  377250  
 Max.   :53676000  

In [ ]:
# Let's remove the "catch_thousands" column for now since we don't neet it.

original_data1 <- original_data1 %>%
  select(-catch_thousands)
head(original_data1)

,Region,Year,species,catch
,<chr>,<int>,<chr>,<dbl>
1,SSE,1886,Chinook,0
2,SSE,1887,Chinook,0
3,SSE,1888,Chinook,0
4,SSE,1889,Chinook,0
5,SSE,1890,Chinook,0
6,SSE,1891,Chinook,0


#Group by and summarise

In [ ]:
original_data1 %>%
  group_by(Region) %>%
  summarise(mean(catch))

Region,mean(catch)
<chr>,<dbl>
ALU,40383.91
BER,16372.55
BRB,2709796.49
CHG,315487.27
CKI,683571.43
COP,179223.40
GSE,133841.46
KOD,1528350.00
KSK,67642.35


In [ ]:
original_data1 %>%
  group_by(species) %>%
  summarise(max(catch))

species,max(catch)
<chr>,<dbl>
Chinook,609000
Chum,10459000
Coho,3220000
Pink,53676000
Sockeye,44269000


In [ ]:
# ...to count the number of rows in each group

original_data1 %>%
  group_by(Region) %>%
  summarize(n=n())

Region,n
<chr>,<int>
ALU,435
BER,510
BRB,570
CHG,550
CKI,525
COP,470
GSE,410
KOD,580
KSK,425


#Filtering rows

In [ ]:
original_data1 %>%
  filter(Region == 'SSE') %>%
  head(12)

,Region,Year,species,catch
,<chr>,<int>,<chr>,<dbl>
1,SSE,1886,Chinook,0
2,SSE,1887,Chinook,0
3,SSE,1888,Chinook,0
4,SSE,1889,Chinook,0
5,SSE,1890,Chinook,0
6,SSE,1891,Chinook,0
7,SSE,1892,Chinook,0
8,SSE,1893,Chinook,0
9,SSE,1894,Chinook,0


In [ ]:
original_data1 %>%
  filter(Region == 'SSE' & species == 'Coho') %>%
  head(12)

,Region,Year,species,catch
,<chr>,<int>,<chr>,<dbl>
1,SSE,1886,Coho,0
2,SSE,1887,Coho,0
3,SSE,1888,Coho,16000
4,SSE,1889,Coho,11000
5,SSE,1890,Coho,42000
6,SSE,1891,Coho,24000
7,SSE,1892,Coho,11000
8,SSE,1893,Coho,1000
9,SSE,1894,Coho,5000


#Sorting your data

In [ ]:
#agrupa el conjunto de datos por la columna "Region", calcula el promedio del
# "catch" para cada región y luego ordena los resultados en función de este promedio.

original_data1 %>%
  group_by(Region) %>%
  summarise(mean_catch = mean(catch)) %>%
  arrange(mean_catch)

Region,mean_catch
<chr>,<dbl>
BER,16372.55
KTZ,18836.14
ALU,40383.91
NRS,51502.70
KSK,67642.35
YUK,68645.57
YAK,91922.92
GSE,133841.46
COP,179223.40


In [ ]:
# The default sorting order of "arrange()" is to sort in ascending order;
# to reverse the sort order:

original_data1 %>%
  group_by(Region) %>%
  summarise(mean_catch = mean(catch)) %>%
  arrange(desc(mean_catch))

Region,mean_catch
<chr>,<dbl>
SSE,3184660.71
BRB,2709796.49
NSE,1825020.87
KOD,1528350.00
PWS,1419236.70
SOP,1110942.22
CKI,683571.43
CHG,315487.27
NOP,229493.48


#Joins

In [ ]:
#Manipulating more than one dataframe

In [ ]:
#second data frame:

region_defs <- read.csv(url("https://knb.ecoinformatics.org/knb/d1/mn/v2/object/df35b.303.1", method = "libcurl"),
                        stringsAsFactors = FALSE)
head(region_defs)

,code,mgmtArea,areaClass,regionCode,notes
,<chr>,<chr>,<chr>,<int>,<chr>
1,GSE,Unallocated Southeast Alaska,mgmtArea,1,Included are Southeastern Alaska catches which could not be assigned to Northern or Southern Southeastern Alaska or Yakutat
2,NSE,Northern Southeast Alaska,mgmtArea,1,Northern Southern Alaska includes Districts 9 through 16 from summer straight northwest to and including the fairweather Grounds but excludes Yakutat fisheries unless otherwise noted.
3,SSE,Southern Southeast Alaska,mgmtArea,1,
4,YAK,Yakutat,mgmtArea,1,
5,PWSmgmt,Prince William Sound Management Area,mgmtArea,2,Includes the Bering River and Copper River and Prince William Sound Subareas
6,BER,Bering River Subarea Copper River Subarea,subarea,2,


In [ ]:
#We can see that the column names don't exactly match in both dataframe


# LEFT JOIN entre tabla REGION_DEFS y ORIGNAL_DATA1
original_data1 %>%
  group_by(Region) %>%
  summarise(total_catch = sum(catch)) %>%
  left_join(region_defs, by = c("Region" = "code"))

Region,total_catch,mgmtArea,areaClass,regionCode,notes
<chr>,<dbl>,<chr>,<chr>,<int>,<chr>
ALU,17567000,Aleutian Islands Subarea,subarea,4,
BER,8350000,Bering River Subarea Copper River Subarea,subarea,2,
BRB,1544584000,Bristol Bay Management Area,mgmtArea,2,
CHG,173518000,Chignik Management Area,mgmtArea,4,
CKI,358875000,Cook Inlet Management Area,mgmtArea,2,Cook Inlet includes Resurrection Bay unless otherwise noted
COP,84235000,Copper River Subarea,subarea,2,
GSE,54875000,Unallocated Southeast Alaska,mgmtArea,1,Included are Southeastern Alaska catches which could not be assigned to Northern or Southern Southeastern Alaska or Yakutat
KOD,886443000,Kodiak Management Area,mgmtArea,4,
KSK,28748000,Kuskokwim Management Area,mgmtArea,3,


In [ ]:
#Another way is to use RENAME to change the column name CODE to REGION in the REGION_DEFS dataframe,
# and run the left_join this way:

region_defs <- region_defs %>%
  rename(Region = code, Region_Name = mgmtArea )

original_data1 %>%
  group_by(Region) %>%
  summarise(total_catch = sum(catch)) %>%
  left_join(region_defs, by = c("Region"))

Region,total_catch,Region_Name,areaClass,regionCode,notes
<chr>,<dbl>,<chr>,<chr>,<int>,<chr>
ALU,17567000,Aleutian Islands Subarea,subarea,4,
BER,8350000,Bering River Subarea Copper River Subarea,subarea,2,
BRB,1544584000,Bristol Bay Management Area,mgmtArea,2,
CHG,173518000,Chignik Management Area,mgmtArea,4,
CKI,358875000,Cook Inlet Management Area,mgmtArea,2,Cook Inlet includes Resurrection Bay unless otherwise noted
COP,84235000,Copper River Subarea,subarea,2,
GSE,54875000,Unallocated Southeast Alaska,mgmtArea,1,Included are Southeastern Alaska catches which could not be assigned to Northern or Southern Southeastern Alaska or Yakutat
KOD,886443000,Kodiak Management Area,mgmtArea,4,
KSK,28748000,Kuskokwim Management Area,mgmtArea,3,


#Split a single column into numerous (or numerous into a single)

In [ ]:
#Let's make a new dataframe with fake data to illustrate

dates_df <- data.frame(date = c("5/24/1930",
                                "5/25/1930",
                                "5/26/1930",
                                "5/27/1930",
                                "5/28/1930"),
                       stringsAsFactors = FALSE)

dates_df %>%
  separate(date, c("month", "day", "year"), "/")

month,day,year
<chr>,<chr>,<chr>
5,24,1930
5,25,1930
5,26,1930
5,27,1930
5,28,1930


In [ ]:
cities_df <- data.frame(city = c("Juneau AK",
                                 "Sitka AK",
                                 "Anchorage AK"),
                        stringsAsFactors = FALSE)

#Split the city column in the following dataframe into city and state_code columns:
cities_df %>%
  separate(city, c("city", "state_code"), " ")

city,state_code
<chr>,<chr>
Juneau,AK
Sitka,AK
Anchorage,AK


In [ ]:
#unite() does just the reverse of separate():

dates_df %>%
  separate(date, c("month", "day", "year"), "/") %>%
  unite(date, month, day, year, sep="/")


date
<chr>
5/24/1930
5/25/1930
5/26/1930
5/27/1930
5/28/1930
